1- Preprocessing :
define characters accepted and tashkeel accepted and remove from the training set any tashkeel and unwanted characters (eg. brackets, numbers... etc)

In [3]:
import re
import numpy as np
import pyarabic.araby as araby
import nltk
from nltk.tokenize import sent_tokenize
import tensorflow as tf
from keras_preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Embedding, LSTM, Dense, SpatialDropout1D
from gensim.models import Word2Vec
# from tensorflow.keras.models import Sequential
# from tensorflow.keras.layers import Dense, Embedding, LSTM, SpatialDropout1D
# from tensorflow.keras.callbacks import EarlyStopping

In [16]:
arabicDictionary=['ى', 'ع', 'ظ', 'ح', 'ر', 'س', 'ي', 'ش', 'ض', 'ق', ' ', 'ث', 'ل', 'ص', 'ط', 'ك', 'آ', 'م', 'ا', 'إ', 'ه', 'ز', 'ء', 'أ', 'ف', 'ؤ', 'غ', 'ج', 'ئ', 'د', 'ة', 'خ', 'و', 'ب', 'ذ', 'ت', 'ن']
punctuations = ["،", ":", "؛", "-", "؟"]
#reading the training dataset
f = open(r"train.txt", "r",encoding="utf-8").read()
print(type(f))
print(len(f))

<class 'str'>
18515271


In [7]:
def cleanDataFromNonArabicLetters(data):
    #regex to keep arabic letters only and remove any other character (eg. brackets, numbers ...etc)
    characters_regex =r'[\s\.\u0600-\u06ff\u0750-\u077f\ufb50-\ufbc1\ufbd3-\ufd3f\ufd50-\ufd8f\ufd50-\ufd8f\ufe70-\ufefc\uFDF0-\uFDFD]+'
    processedData = re.findall(characters_regex,data)
    processedData = " ".join(processedData)
    processedData = re.sub(r"\s+"," " ,processedData) #substitute many spaces with one space only
    return processedData

In [8]:
processedData=cleanDataFromNonArabicLetters(f)
print(len(processedData))

17997403


In [9]:
def removeDiacratics(processedData):
    without_diacritics= araby.strip_diacritics(processedData)
    return without_diacritics

In [10]:
without_diacritics = removeDiacratics(processedData)
print(len(without_diacritics))

10711561


In [15]:
#take the procrssed text and separate it into sentences
def generateLists(with_diacratics, without_diacritics):
    #generate longStringSplited which is the list of procrssed text without practices and numbers and dots 
    #the rest of the punctuation still there
    longStringWithDecimalPoint=re.sub(r"\n", ' ', with_diacratics) #string of data with diacratics removed endlines from it
    longStringSplited=longStringWithDecimalPoint.split('.') #split the string by dots (segment sentences) and make list of them
    longString=' '.join(longStringSplited)  #long string without decimal points
  
    without_diacritics_new=re.sub(r"\n", ' ', without_diacritics) #string of data without diacratics removed endlines from it
    without_diacritics_splitted=without_diacritics_new.split('.') #split the string by dots (segment sentences) and make list of them

    list_of_sentences=[]
    for line in without_diacritics_splitted: #list of lists of sentences splitted in words without diacratics
        list_of_sentences.append(line.split(" "))
    
    for element in punctuations:
        longString=re.sub(element, '', longString) #remove punctuations from the string

'''
COMPLETE THE FUNCTION AND TEST THE LISTS
'''


40836
['قَوْلُهُ أَوْ قَطَعَ الْأَوَّلُ يَدَهُ إلَخْ قَالَ الزَّرْكَشِيُّ ابْنُ عَرَفَةَ قَوْلُهُ بِلَفْظٍ يَقْتَضِيه كَإِنْكَارِ غَيْرِ حَدِيثٍ بِالْإِسْلَامِ وُجُوبَ مَا عُلِمَ وُجُوبُهُ مِنْ الدِّينِ ضَرُورَةً كَإِلْقَاءِ مُصْحَفٍ بِقَذَرٍ وَشَدِّ زُنَّارٍ ابْنُ عَرَفَةَ قَوْلُ ابْنِ شَاسٍ أَوْ بِفِعْلٍ يَتَضَمَّنُهُ هُوَ كَلُبْسِ الزُّنَّارِ وَإِلْقَاءِ الْمُصْحَفِ فِي صَرِيحِ النَّجَاسَةِ وَالسُّجُودِ لِلصَّنَمِ وَنَحْوِ ذَلِكَ وَسِحْرٍ مُحَمَّدٌ قَوْلُ مَالِكٍ وَأَصْحَابِهِ أَنَّ السَّاحِرَ كَافِرٌ بِاَللَّهِ تَعَالَى قَالَ مَالِكٌ هُوَ كَالزِّنْدِيقِ إذَا عَمِلَ السِّحْرَ بِنَفْسِهِ قُتِلَ وَلَمْ يُسْتَتَبْ ', ' قَوْلُهُ لِعَدَمِ مَا تَتَعَلَّقُ إلَخْ أَيْ الْوَصِيَّةُ قَوْلُهُ مَا مَرَّ أَيْ قُبَيْلَ قَوْلِ الْمَتْنِ لَغَتْ وَلَوْ اقْتَصَرَ عَلَى أَوْصَيْت لَهُ بِشَاةٍ أَوْ أَعْطُوهُ شَاةً وَلَا غَنَمَ لَهُ عِنْدَ الْمَوْتِ هَلْ تَبْطُلُ الْوَصِيَّةُ أَوْ يُشْتَرَى لَهُ شَاةٌ وَيُؤْخَذُ مِنْ قَوْلِهِ الْآتِي كَمَا لَوْ لَمْ يَقُلْ مِنْ مَالِي وَلَا مِنْ غَنَمِي أَنَّهَا لَا تَب

In [9]:
# now the varable called longString has a single string with all the processed words in it 
listOfwordsWith_Diacritics=list()
listOfwordsWith_NoDiacritics=list()

without_diacritics= araby.strip_diacritics(longString)


without_diacritics=re.sub(r"\s+", ' ', without_diacritics)

listOfwordsWith_NoDiacritics=without_diacritics.split(" ")


listOfwordsWith_Diacritics=re.sub(r"\s+", ' ', longString)
listOfwordsWith_Diacritics=listOfwordsWith_Diacritics.split(" ")





In [10]:
#now that we have two separated lists we need to get the diacritics list 


listofDiacritrcs_ToWord=list()
temp=list()
counter=0
flag=0
for word in listOfwordsWith_Diacritics:
    while counter<len(word):
     if word[counter] in arabicDictionary: #checking if the character is a letter
      if (counter+1)<len(word):
        #checking if the next character is also a letter, then that means that the diacritics of the current letter is none so add empty string to the list
        if word[counter +1] in arabicDictionary: 
          temp.append("")
          counter+=1
          continue
      counter+=1 #if it is the end of the word (no more letters) or the next character is a diacritics -> continue looping
      continue
     else:
      if (counter+1)<len(word):
        if word[(counter+1)] not in arabicDictionary: #if the current and the next characters are diacritics, add them together in the list
          temp.append(word[counter]+word[counter+1])
          counter+=2
          continue
      temp.append(word[counter]) #if the current character only is the diacritics add it to the list
      counter+=1    
    listofDiacritrcs_ToWord.append(temp.copy())     
    temp.clear() 
    counter=0
          
          
    
        


print(listOfwordsWith_Diacritics[0:100])
print(listOfwordsWith_NoDiacritics[0:100])

['قَوْلُهُ', 'أَوْ', 'قَطَعَ', 'الْأَوَّلُ', 'يَدَهُ', 'إلَخْ', 'قَالَ', 'الزَّرْكَشِيُّ', 'ابْنُ', 'عَرَفَةَ', 'قَوْلُهُ', 'بِلَفْظٍ', 'يَقْتَضِيه', 'كَإِنْكَارِ', 'غَيْرِ', 'حَدِيثٍ', 'بِالْإِسْلَامِ', 'وُجُوبَ', 'مَا', 'عُلِمَ', 'وُجُوبُهُ', 'مِنْ', 'الدِّينِ', 'ضَرُورَةً', 'كَإِلْقَاءِ', 'مُصْحَفٍ', 'بِقَذَرٍ', 'وَشَدِّ', 'زُنَّارٍ', 'ابْنُ', 'عَرَفَةَ', 'قَوْلُ', 'ابْنِ', 'شَاسٍ', 'أَوْ', 'بِفِعْلٍ', 'يَتَضَمَّنُهُ', 'هُوَ', 'كَلُبْسِ', 'الزُّنَّارِ', 'وَإِلْقَاءِ', 'الْمُصْحَفِ', 'فِي', 'صَرِيحِ', 'النَّجَاسَةِ', 'وَالسُّجُودِ', 'لِلصَّنَمِ', 'وَنَحْوِ', 'ذَلِكَ', 'وَسِحْرٍ', 'مُحَمَّدٌ', 'قَوْلُ', 'مَالِكٍ', 'وَأَصْحَابِهِ', 'أَنَّ', 'السَّاحِرَ', 'كَافِرٌ', 'بِاَللَّهِ', 'تَعَالَى', 'قَالَ', 'مَالِكٌ', 'هُوَ', 'كَالزِّنْدِيقِ', 'إذَا', 'عَمِلَ', 'السِّحْرَ', 'بِنَفْسِهِ', 'قُتِلَ', 'وَلَمْ', 'يُسْتَتَبْ', 'قَوْلُهُ', 'لِعَدَمِ', 'مَا', 'تَتَعَلَّقُ', 'إلَخْ', 'أَيْ', 'الْوَصِيَّةُ', 'قَوْلُهُ', 'مَا', 'مَرَّ', 'أَيْ', 'قُبَيْلَ', 'قَوْلِ', 'الْمَتْنِ', 'لَغَتْ', 'وَلَوْ', 'اقْت

In [11]:
print(len(listOfwordsWith_NoDiacritics))
print(listOfwordsWith_Diacritics[3])
print(listOfwordsWith_NoDiacritics[3])
print(listofDiacritrcs_ToWord[3])
print(" ".join(listofDiacritrcs_ToWord[3]))
print(listofDiacritrcs_ToWord[0:100])

2101285
الْأَوَّلُ
الأول
['', 'ْ', 'َ', 'َّ', 'ُ']
 ْ َ َّ ُ
[['َ', 'ْ', 'ُ', 'ُ'], ['َ', 'ْ'], ['َ', 'َ', 'َ'], ['', 'ْ', 'َ', 'َّ', 'ُ'], ['َ', 'َ', 'ُ'], ['', 'َ', 'ْ'], ['َ', '', 'َ'], ['', '', 'َّ', 'ْ', 'َ', 'ِ', 'ُّ'], ['', 'ْ', 'ُ'], ['َ', 'َ', 'َ', 'َ'], ['َ', 'ْ', 'ُ', 'ُ'], ['ِ', 'َ', 'ْ', 'ٍ'], ['َ', 'ْ', 'َ', 'ِ', ''], ['َ', 'ِ', 'ْ', 'َ', '', 'ِ'], ['َ', 'ْ', 'ِ'], ['َ', 'ِ', '', 'ٍ'], ['ِ', '', 'ْ', 'ِ', 'ْ', 'َ', '', 'ِ'], ['ُ', 'ُ', '', 'َ'], ['َ'], ['ُ', 'ِ', 'َ'], ['ُ', 'ُ', '', 'ُ', 'ُ'], ['ِ', 'ْ'], ['', '', 'ِّ', '', 'ِ'], ['َ', 'ُ', '', 'َ', 'ً'], ['َ', 'ِ', 'ْ', 'َ', '', 'ِ'], ['ُ', 'ْ', 'َ', 'ٍ'], ['ِ', 'َ', 'َ', 'ٍ'], ['َ', 'َ', 'ِّ'], ['ُ', 'َّ', '', 'ٍ'], ['', 'ْ', 'ُ'], ['َ', 'َ', 'َ', 'َ'], ['َ', 'ْ', 'ُ'], ['', 'ْ', 'ِ'], ['َ', '', 'ٍ'], ['َ', 'ْ'], ['ِ', 'ِ', 'ْ', 'ٍ'], ['َ', 'َ', 'َ', 'َّ', 'ُ', 'ُ'], ['ُ', 'َ'], ['َ', 'ُ', 'ْ', 'ِ'], ['', '', 'ُّ', 'َّ', '', 'ِ'], ['َ', 'ِ', 'ْ', 'َ', '', 'ِ'], ['', 'ْ', 'ُ', 'ْ', 'َ', 'ِ'], ['ِ'], ['َ', 'ِ', '', 'ِ'],

2-Word Embeddings

In [14]:
# Train Word2Vec model
Word2Vec_model = Word2Vec(list_of_sentences, vector_size=100, window=5, min_count=1, workers=4, sg=1)
Word2Vec_model.train(list_of_sentences,total_examples=len(listOfwordsWith_NoDiacritics),epochs=10)

(100000, 21012850)

In [16]:
word_embeddings = {word: Word2Vec_model.wv[word] for word in Word2Vec_model.wv.index_to_key}

3-Building the LSTM model for character level classifier

In [17]:
for i in range(len(listofDiacritrcs_ToWord)):
        listofDiacritrcs_ToWord[i] = " ".join(listofDiacritrcs_ToWord[i])
# print(listofDiacritrcs_ToWord[0:100])
Y_train= np.array([listofDiacritrcs_ToWord],dtype=object).T
X_train = np.array([listOfwordsWith_NoDiacritics],dtype=object)
print(str(X_train.shape))

(1, 2101285)


In [18]:
# Tokenize the input words and diacritics
word_tokenizer = tf.keras.preprocessing.text.Tokenizer(char_level=True, filters='')
word_tokenizer.fit_on_texts(listOfwordsWith_NoDiacritics[0:50000])
word_sequences = word_tokenizer.texts_to_sequences(listOfwordsWith_NoDiacritics[0:50000])

diacritic_tokenizer = tf.keras.preprocessing.text.Tokenizer(char_level=True, filters='')
diacritic_tokenizer.fit_on_texts([''.join(d) for d in listofDiacritrcs_ToWord[0:50000]])
diacritic_sequences = diacritic_tokenizer.texts_to_sequences([''.join(d) for d in listofDiacritrcs_ToWord[0:50000]])

# Pad sequences to have the same length
max_len = max(max(len(seq) for seq in word_sequences), max(len(seq) for seq in diacritic_sequences))
padded_word_sequences = pad_sequences(word_sequences, maxlen=max_len, padding='post')
padded_diacritic_sequences = pad_sequences(diacritic_sequences, maxlen=max_len, padding='post')

# Build the LSTM model
model = Sequential()
model.add(Embedding(input_dim=len(word_tokenizer.word_index) + 1, output_dim=50, input_length=max_len))
model.add(LSTM(100, return_sequences=True))
model.add(Dense(len(diacritic_tokenizer.word_index) + 1, activation='softmax'))

# Compile the model
model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# Train the model
model.fit(padded_word_sequences, np.expand_dims(padded_diacritic_sequences, -1), epochs=10, batch_size=32)

# Save the model for later use
model.save("diacritic_prediction_model.h5")

# Now, you can use the trained model to predict diacritics for new Arabic words
def predict_diacritics(model, word):
    word_sequence = word_tokenizer.texts_to_sequences([word])
    padded_word_sequence = pad_sequences(word_sequence, maxlen=max_len, padding='post')
    predicted_diacritic_sequence = model.predict(padded_word_sequence)
    predicted_diacritic_sequence = np.argmax(predicted_diacritic_sequence, axis=-1)
    predicted_diacritic = diacritic_tokenizer.sequences_to_texts(predicted_diacritic_sequence)
    return predicted_diacritic[0]




Epoch 1/10
1563/1563 [==============================] - 16s 9ms/step - loss: 0.4726 - accuracy: 0.8458
Epoch 2/10
1563/1563 [==============================] - 15s 9ms/step - loss: 0.2472 - accuracy: 0.9151
Epoch 3/10
1563/1563 [==============================] - 14s 9ms/step - loss: 0.2055 - accuracy: 0.9292
Epoch 4/10
1563/1563 [==============================] - 14s 9ms/step - loss: 0.1844 - accuracy: 0.9358
Epoch 5/10
1563/1563 [==============================] - 15s 9ms/step - loss: 0.1707 - accuracy: 0.9398
Epoch 6/10
1563/1563 [==============================] - 15s 10ms/step - loss: 0.1607 - accuracy: 0.9427
Epoch 7/10
1563/1563 [==============================] - 15s 9ms/step - loss: 0.1534 - accuracy: 0.9449
Epoch 8/10
1563/1563 [==============================] - 15s 9ms/step - loss: 0.1473 - accuracy: 0.9468
Epoch 9/10
1563/1563 [==============================] - 15s 10ms/step - loss: 0.1421 - accuracy: 0.9483
Epoch 10/10
1563/1563 [==============================] - 15s 10ms/step 

In [26]:
# Example usage
# input_word = "الذي علم بالقلم"
# input_word = "ذهب علي إلى الشاطئ"
input = "قال الزركشي ابن عرفة قوله بلفظ يقتضيه كإنكار غير حديث بالإسلام وجوب ما علم وجوبه من الدين ضرورة كإلقاء مصحف بقذر"
input_word = input.split(' ')
word_vectors = [Word2Vec_model.wv[word] for word in input_word if word in Word2Vec_model.wv]
# print(word_vectors)

for word in input_word: 
  predicted_diacritics1 = predict_diacritics(model, word)
  print(list(predicted_diacritics1))
  print(f"Input word: {word}")
  print(f"Predicted diacritics: {predicted_diacritics1}")
  tempString=str()
  counterWord=0
  counterDiacrtic=0
  while counterDiacrtic<(len(predicted_diacritics1)):
    if predicted_diacritics1[counterDiacrtic]!=' ':
      tempString=tempString+word[counterWord]
      tempString=tempString+predicted_diacritics1[counterDiacrtic]
      print(predicted_diacritics1[counterDiacrtic])
      if predicted_diacritics1[counterDiacrtic] == 'ّ':
        print("here")
        tempString=tempString+predicted_diacritics1[counterDiacrtic+2]
        counterDiacrtic=counterDiacrtic+2
      counterWord+=1
    counterDiacrtic+=1

  print(tempString)

1/1 [==============================] - 0s 19ms/step
['َ', ' ', ' ', ' ', ' ', ' ', 'َ']
Input word: قال
Predicted diacritics: َ     َ
َ
َ
قَاَ
1/1 [==============================] - 0s 19ms/step
[' ', ' ', 'ْ', ' ', 'ّ', ' ', 'َ', ' ', ' ', ' ', 'ْ', ' ', ' ', ' ', 'َ', ' ', ' ', ' ', 'ِ', ' ', ' ', ' ', 'ّ', ' ', 'ُ']
Input word: الزركشي
Predicted diacritics:   ْ ّ َ   ْ   َ   ِ   ّ ُ
ْ
ّ
here
ْ
َ
ِ
ّ
here
اْلَّزْرَكِشُّ
1/1 [==============================] - 0s 17ms/step
[' ', ' ', 'ْ', ' ', ' ', ' ', 'ِ']
Input word: ابن
Predicted diacritics:   ْ   ِ
ْ
ِ
اْبِ
1/1 [==============================] - 0s 17ms/step
['َ', ' ', ' ', ' ', 'َ', ' ', ' ', ' ', 'َ', ' ', ' ', ' ', 'َ']
Input word: عرفة
Predicted diacritics: َ   َ   َ   َ
َ
َ
َ
َ
عَرَفَةَ
1/1 [==============================] - 0s 17ms/step
['َ', ' ', ' ', ' ', 'ْ', ' ', ' ', ' ', 'ُ', ' ', ' ', ' ', 'ُ']
Input word: قوله
Predicted diacritics: َ   ْ   ُ   ُ
َ
ْ
ُ
ُ
قَوْلُهُ
1/1 [==============================] - 0s 17ms/step
['ِ